In [1]:
import numpy as np
from tqdm import tqdm
from PIL import Image
import tensorflow as tf
import glob

from collections import defaultdict
import random
import copy
from tensorflow import keras
from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.layers import Input, Dense, Dropout, Lambda, Activation, concatenate, LSTM
from tensorflow.keras.models import Model
import tensorflow.keras.optimizers
from keras.utils import np_utils
#from tensorflow.keras import backend as K
#K.tensorflow_backend._get_available_gpus()
tf.test.is_gpu_available(cuda_only=True)

try:
    os.mkdir('results/3sampleclassification/')
except:
    pass

Using TensorFlow backend.


In [2]:
height,width=(540,540)


def getvideoname(name):
    return '_'.join(name.split('/')[-1].split('_')[:3])

def getframe(name):
    return name.split('/')[-1].split('_')[-1].split('.')[0]

def getlabel(name):
    if len(name.split('/')[-1].split('_'))>=5:
        try:
            return [float(x) for x in name.split('/')[-1].split('_')[3:-1]]
        except:
            return [0,0,0]
    else:
        return [0,0,0]


def load_data():
    dataset_path = 'trainingdata/'
    frames=glob.glob(dataset_path+'**/*frames_face/*.*.jpg')
    indexes=[x for x in range(len(frames))]
    datadict=defaultdict(list)
    labelsdict={}
    for frame in frames:
        datadict[getvideoname(frame)].append(frame)
    for key,data in datadict.items():
        #print(key)
        datadict[key] = sorted(data, key=lambda x:int(getframe(x)))
        labelsdict[key]=[getlabel(x) for x in data]
    #random.shuffle(indexes)
    return indexes,datadict,labelsdict

def find_index(index,data,labels):
    for key in data.keys():
        if index>=len(data[key]) - 2:
            index-=len(data[key])
            continue
        return data[key][index],labels[key][index]


def get_img(index, indexes, X, Y):
    x,y=find_index(index,X,Y)
    if Y==[0,0,0]:
        x,y=find_index(random.randint(len(indexes),X,Y))
    #print(x)
    x=Image.open(x)
    x.convert('RGB')
    x=np.array(x)
    #x.resize((224,224,3))
    return x,y

def data_generator(X,Y,indexes,batch_size=12):
    while True:
        idxs=np.random.permutation(len(indexes))
        p0,p1,p2, q = [],[],[],[]
        for i in range(len(X) - 1):
            x_2, y_2 = get_img(idxs[i]+2, indexes, X, Y)
            # while y_1==[0,0,0]:
            #     tempindex=random.randint(len(indexes)-1)
            #     x_1, y_1 = get_img(temp+1, indexes, X, Y)
            # x,y = get_img(temp, indexes, X, Y)
            x_1, y_1 = get_img(idxs[i]+1, indexes, X, Y)
            x,y = get_img(idxs[i], indexes, X, Y)
            #while y_2==[0,0,0]:
            #    tempindex=random.randint(len(indexes)-2)
            #    x_2, y_2 = get_img(tempindex+2, indexes, X, Y)
            #    x_1, y_1 = get_img(tempindex+1, indexes, X, Y)
            #    x,y = get_img(tempindex+1, indexes, X, Y)
            

            p0.append(x)
            p1.append(x_1)
            p2.append(x_2)
            #print(str(y_2[0])+' '+str(converttolabel(y_2[0])))
            q.append(converttolabel(y_2[1]))
            if len(q)==batch_size:
                yield {'input1': np.array(p0), 'input2': np.array(p1),'input3':np.array(p2)}, np.array(q)
                p0, p1,p2, q=[],[], [],[]
        if p0:
            yield {'input1': np.array(p0), 'input2': np.array(p1),'input3':np.array(p2)}, np.array(q)
            p0, p1,p2, q=[],[],[], []

def custom_stack_layer(tensor):
    return tf.keras.backend.stack(
        tensor, axis=1
    )

def converttolabel(num):
    #return int((num+1.1912906024096384)*10)
    return int((num+0.24397519277108434)*10)

In [3]:
_,_l,labeldict=load_data()
#print(labeldict)
nums,avg,minnum,maxnum=0,0,10000,-10000
for x,labels in labeldict.items():
    if min([i[1] for i in labels])<minnum: minnum=min([i[1] for i in labels])
    if max([i[1] for i in labels])>maxnum: maxnum=max([i[1] for i in labels])  
    for label in labels:
        
        if label==[0,0,0]:
            continue
        else:
            nums+=1
            avg+=label[1]
avg/=nums
print(nums)
print(avg)
print(minnum)
print(maxnum)

38341
0.9413920541894304
-0.24397519277108434
5.768446506024094


In [4]:
indexes,data,labels=load_data()
randomprints=open('random.txt','w')
#randomprints.write(len(indexes))

input_image=Input(shape=(height,width,3), name='input1')
input_image2=Input(shape=(height,width,3), name='input2')
input_image3=Input(shape=(height,width,3), name='input3')
randomprints.write(str(input_image.shape))
randomprints.flush()
mobilenet = MobileNet(include_top=False,pooling='avg')
x1 = mobilenet(input_image)
x2 = mobilenet(input_image2)
x3= mobilenet(input_image3)
x = Lambda(custom_stack_layer, name="lambda_layer")([x1,x2,x3])

lstm = LSTM(
    units=256, return_sequences=False, dropout=0.2
)(x)

out = Dense(60, activation='relu', trainable=False, name='out')(lstm)

predict=Activation(activation='softmax')(out)

model=Model(inputs=[input_image, input_image2,input_image3], outputs=predict)
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy', metrics=['accuracy'])

train_indexes=indexes[:int(0.7*len(indexes))]
validation_indexes=indexes[int(0.7*len(indexes)):]

earlystop = tf.keras.callbacks.EarlyStopping( monitor = 'val_acc', patience=5 )
reducelr = tf.keras.callbacks.ReduceLROnPlateau( monitor = 'val_acc', patience=2, factor=0.2, min_lr=0.0001 )

history=model.fit_generator(data_generator(data,labels,train_indexes),
                            steps_per_epoch=800,
                            epochs=50,
                            validation_data=data_generator(data,labels,validation_indexes),
                            validation_steps=385,
                            callbacks=[earlystop,reducelr])




import pandas as pd
hist_df=pd.DataFrame(history.history)



Instructions for updating:
If using Keras pass *_constraint arguments to layers.


/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras_applications/mobilenet.py:207: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '


Epoch 1/50
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
800/800 [==============================] - 537s 672ms/step - loss: 2.3478 - acc: 0.4370 - val_loss: 2.4091 - val_acc: 0.3551
Epoch 2/50
800/800 [==============================] - 519s 648ms/step - loss: 1.6875 - acc: 0.5511 - val_loss: 2.0324 - val_acc: 0.4577
Epoch 3/50
800/800 [==============================] - 520s 650ms/step - loss: 1.6570 - acc: 0.5288 - val_loss: 1.9814 - val_acc: 0.4623
Epoch 4/50
800/800 [==============================] - 518s 648ms/step - loss: 1.6279 - acc: 0.5253 - val_loss: 1.9956 - val_acc: 0.4259
Epoch 5/50
800/800 [==============================] - 522s 652ms/step - loss: 1.5477 - acc: 0.5558 - val_loss: 1.9336 - val_acc: 0.4373
Epoch 6/50
800/800 [==============================] - 516s 645ms/step - loss: 1.4679 - acc: 0.5531 - val_loss: 1.7103 - val_acc: 0.4392
Epoch 7/50
800/800 [==============================] - 516s 645ms/step - loss: 1.4208 - acc

FileNotFoundError: [Errno 2] No such file or directory: 'results/3sampleclassification/hist.csv'

In [5]:
with open("histlabel1.csv",mode='w') as f:
    hist_df.to_json(f)


model_json=model.to_json()
with open("MobileNetTransferlabel1.json",'w') as json_file:
    json_file.write(model_json)

model.save_weights('MobileNetTransferlabel1.h5')